In [36]:
import numpy as np
from matplotlib import pyplot as plt
import os
from astropy.table import Table
from astropy.wcs import WCS

from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii

In [37]:
homedir = os.getenv("HOME")

vfmain = Table.read(homedir+'/sample_main.fits')
vfz0mgs = Table.read(homedir+'/sample_mgs.fits')

In [38]:
def readfile(filename):                                      #function that converts that wonky string
                                                             #output txt file to a usable set of lists
                                                             #that I can manipulate with indices
    
    fileobj=open(filename,'r')                               #opens the file, of course
    words=fileobj.read().splitlines()                        #splits the header and data into two lists
    
    header = words[0].split()                                #splits header, data arrays into elements
    data = words[1].split()
    
    fileobj.close()
    return header,data




def readfile2(filename):                                     #alternative function, if galfit fails...
                                                             #since galfit failing results in no data file,
                                                             #it would be rather nonsensical to define
                                                             #a data list.
    fileobj=open(filename,'r')
    words=fileobj.read().splitlines()
    header = words[0].split()
    fileobj.close()
    return header
    
    
#Here, I effectively create a function that loops through each galaxy element in the input sample.
#Once galfit executes, the code then generates a text file consisting of a header and the data outputs
#representing the modeling parameters. There are also two additional columns: the galaxy prefix, and a
#sucess_flag, which marks whether galfit ran successfully per galaxy.
    
def run_galfit_all(galaxy_sample):
    
    os.chdir(homedir+'/github/WISE_test')
    %run ~/github/virgowise/wisesize.py
    
    
    
    for n in range(0,len(galaxy_sample)):
        
        try:
            g = galaxy(galaxy_sample['RA'][n], galaxy_sample['DEC'][n],
                      galaxy_sample['radius'][n], name = galaxy_sample['prefix'][n], band='3')
            print(galaxy_sample['prefix'][n])
            g.run_simple(convflag=False)
            
            t = homedir+'/github/WISE_test/'+galaxy_sample[n]['prefix']+'-unwise-w3-log.txt'
            header,data = readfile(t)
            header.pop(0)                                    #removes the pound_sign from the array
            header.append('prefix')
            header.append('success_flag')
            
            for i in range(0,len(data)):
                data[i] = float(data[i])
            data.append(galaxy_sample[n]['prefix'])
            data.append(1)                                   #success_flag value of one
            
            if n == 0:                                       #if the galaxy is the first entry, then
                file_test = [header,data]                    #append to the list both the header & data lists
                file_plots = [header,data]                   #append to list for corner plots
                
            else:
                file_test2 = [header,data]                   #otherwise, only include the data list
                file_test.append(file_test2[1])
                file_plots.append(file_test2[1])
                    
        except:
            
            t = homedir+'/github/WISE_test/'+galaxy_sample[n]['prefix']+'-unwise-w3-log.txt'
            header = readfile2(t)
            header.pop(0)                                    #removes the pound_sign from the array
            header.append('prefix')
            header.append('success_flag')
            
            data = []
            for num in range(0,len(header)-2):
                data.append(-999)
            data.append(galaxy_sample[n]['prefix'])
            data.append(0)                                   #success_flag value of zero
               
            file_test2 = [header,data]
            file_test.append(file_test2[1])

            
            print(galaxy_sample['prefix'][n], ' ' , 'was unsuccessful.')
            continue
        
    data_array = np.array(file_test)
    data_array_plots = np.array(file_plots)
    np.savetxt("testest2.txt",data_array,fmt="%s")
    np.savetxt("testest2_plots.txt",data_array_plots,fmt="%s")

In [40]:
tab = ascii.read("testest2.txt")
tab

xc,xc_err,yc,yc_err,mag,mag_err,re,re_err,nsersic,nsrsic_err,BA,BA_err,PA,PA_err,sky,sky_err,error,chi2nu,prefix,success_flag
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,int64
175.3109,0.0111,174.8707,0.0189,4.8494,0.0011,17.237,0.0234,0.3845,0.0013,0.4705,0.0006,30.7007,0.0672,-5.0654,0.9446,0.0,3.4527,VFID0388-NGC6015,1
180.2237,0.0114,179.2617,0.0093,6.4258,0.0152,11.0092,0.2344,2.458,0.0328,0.9349,0.0071,-13.1859,4.4584,-62.2269,1.141,0.0,1.0938,VFID0484-NGC3945,1
168.6167,0.0087,169.1971,0.0075,6.6111,0.0087,7.4944,0.0924,1.9954,0.0186,0.8314,0.0047,87.1794,1.112,-11.8236,0.9699,0.0,1.2262,VFID0525-NGC5322,1
170.5771,0.0154,170.4544,0.012,6.5457,0.0134,9.9726,0.1871,2.016,0.0268,0.7574,0.0062,-87.4048,1.0849,-17.2861,1.2849,0.0,1.4051,VFID0547-NGC2768,1
338.7939,0.0066,337.7264,0.0143,3.394,0.0004,38.0683,0.0214,0.6033,0.0007,0.0965,0.0,-24.7751,0.004,11.9695,0.5393,1.0,2.0628,VFID0934-NGC5907,1
189.5989,0.0079,190.6553,0.0055,5.4851,0.0013,8.4349,0.0154,0.777,0.0028,0.3261,0.0005,-54.4628,0.0439,3.7732,0.918,0.0,1.7804,VFID0974-NGC5866,1
263.8745,0.0028,262.3273,0.0116,3.3039,0.0009,17.7386,0.0227,0.9567,0.003,0.2024,0.0002,-11.9891,0.0154,12.082,1.6175,0.0,4.9164,VFID0983-NGC3079,1
213.6396,0.1069,213.3482,0.059,4.943,0.0045,38.0128,0.1017,0.0915,0.002,0.5439,0.0018,68.6517,0.1861,21.7311,1.2065,0.0,1.8424,VFID1169-MESSIER109,1
197.6238,0.0193,193.3163,0.0457,4.0849,0.0016,31.1593,0.0444,0.2512,0.0011,0.4238,0.0007,12.7613,0.0632,25.1176,1.3942,0.0,2.0217,VFID1215-NGC3953,1


In [41]:
tab[24]

xc,xc_err,yc,yc_err,mag,mag_err,re,re_err,nsersic,nsrsic_err,BA,BA_err,PA,PA_err,sky,sky_err,error,chi2nu,prefix,success_flag
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,int64
-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,VFID2713-NGC4631,0


In [42]:
#galaxy index for which the output text file data is nonexistent.

for i in range(0,len(vfmain)):
    if vfmain[i]['prefix'] == 'VFID2713-NGC4631':
        print(i)

24


THE CULPRIT: VFID2713-NGC4631.

VFMAIN[24]

In [43]:
#second table for plotting purposes (see "secsic_profile_exercises" notebook)
tab2 = ascii.read("testest2_plots.txt")

In [44]:
tab2

xc,xc_err,yc,yc_err,mag,mag_err,re,re_err,nsersic,nsrsic_err,BA,BA_err,PA,PA_err,sky,sky_err,error,chi2nu,prefix,success_flag
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,int64
175.3109,0.0111,174.8707,0.0189,4.8494,0.0011,17.237,0.0234,0.3845,0.0013,0.4705,0.0006,30.7007,0.0672,-5.0654,0.9446,0.0,3.4527,VFID0388-NGC6015,1
180.2237,0.0114,179.2617,0.0093,6.4258,0.0152,11.0092,0.2344,2.458,0.0328,0.9349,0.0071,-13.1859,4.4584,-62.2269,1.141,0.0,1.0938,VFID0484-NGC3945,1
168.6167,0.0087,169.1971,0.0075,6.6111,0.0087,7.4944,0.0924,1.9954,0.0186,0.8314,0.0047,87.1794,1.112,-11.8236,0.9699,0.0,1.2262,VFID0525-NGC5322,1
170.5771,0.0154,170.4544,0.012,6.5457,0.0134,9.9726,0.1871,2.016,0.0268,0.7574,0.0062,-87.4048,1.0849,-17.2861,1.2849,0.0,1.4051,VFID0547-NGC2768,1
338.7939,0.0066,337.7264,0.0143,3.394,0.0004,38.0683,0.0214,0.6033,0.0007,0.0965,0.0,-24.7751,0.004,11.9695,0.5393,1.0,2.0628,VFID0934-NGC5907,1
189.5989,0.0079,190.6553,0.0055,5.4851,0.0013,8.4349,0.0154,0.777,0.0028,0.3261,0.0005,-54.4628,0.0439,3.7732,0.918,0.0,1.7804,VFID0974-NGC5866,1
263.8745,0.0028,262.3273,0.0116,3.3039,0.0009,17.7386,0.0227,0.9567,0.003,0.2024,0.0002,-11.9891,0.0154,12.082,1.6175,0.0,4.9164,VFID0983-NGC3079,1
213.6396,0.1069,213.3482,0.059,4.943,0.0045,38.0128,0.1017,0.0915,0.002,0.5439,0.0018,68.6517,0.1861,21.7311,1.2065,0.0,1.8424,VFID1169-MESSIER109,1
197.6238,0.0193,193.3163,0.0457,4.0849,0.0016,31.1593,0.0444,0.2512,0.0011,0.4238,0.0007,12.7613,0.0632,25.1176,1.3942,0.0,2.0217,VFID1215-NGC3953,1
